In [1]:
# packages
import pandas as pd
import sqlite3
from datetime import timedelta
from dtype_dictionaries import create_dtype_dict
import math

## Metdata imputation
All the data is weather related and has a time dimension. 

#### Numeric values
(a) Sparse missings

Impute by interpolating the time series from the same Station

(b) Dens missings

Impute by taking averaging nearby stations

In [2]:
conn = sqlite3.connect('../data/formatted_zone/formatted_zone.db')
table_name = 'Metoffice_01_22_2023-10-13'
dtype_dict, date_columns = create_dtype_dict(table_name)
df = pd.read_sql_query(f"SELECT * FROM \"{table_name}\";", conn, parse_dates=date_columns)

dtype_dict, date_columns = create_dtype_dict('weather_station_locations_2023-10-20')
loc_df = pd.read_sql_query(f"SELECT * FROM \"{'weather_station_locations_2023-10-20'}\";", conn, parse_dates=date_columns)
conn.close()

Metoffice_01_22_2023-10-13
weather_station_locations_2023-10-20


In [3]:
station = "Aviemore"
column = "WSPD"
i = 4

In [4]:
date = df.loc[4,"Date"]

In [5]:
def euclidean_distance(lat1, lon1, lat2, lon2):
    # Assuming the Earth is a flat plane, calculate the distance using Pythagoras' theorem
    lat_diff = lat2 - lat1
    lon_diff = lon2 - lon1
    distance = math.sqrt(lat_diff**2 + lon_diff**2)

    return distance

In [6]:
lat = loc_df.loc[loc_df.SITE == station].LAT
lon = loc_df.loc[loc_df.SITE == station].LON

In [10]:
loc_df['Distance'] = loc_df.apply(lambda row: euclidean_distance(lat, lon, row['LAT'], row['LON']), axis=1)

# Sort the DataFrame by distance
loc_df = loc_df.sort_values(by='Distance')

# Select the top three closest stations
closest_stations = loc_df.head(4)


In [13]:
closest_stations = list(closest_stations.SITE)

In [15]:
df.Station_name in closest_stations

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [14]:
df.loc[(df.Date == date) & (df.Station_name in closest_stations)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
interpolation_threshold = 0.8
station_counts = df.Station_name.value_counts() / 2

for column in df.columns:
    # count null values per station for this column
    station_null_counts = df.loc[df.loc[:,column].isna()].Station_name.value_counts()
    
    # iterate through the dataframe
    for i, value in enumerate(df[column]):
        if pd.isna(value):
            # handle the null value
            station = df.loc[i, "Station_name"]

            # check if this station should be interpolated for this column
            perc_missing = station_null_counts[station] / station_counts[station]
            if  perc_missing < interpolation_threshold:
                # interpolate by the station over days
                
                date = df.iloc[i].Date
                start_date = date - timedelta(days = 2)
                end_date = date + timedelta(days = 2)
                
                interpolation_df = df.loc[(df.Station_name == station) & 
                          (df.Date.dt.time == date.time()) & 
                          (df.Date >= start_date) &
                          (df.Date <= end_date)]
                
                # TODO change to interpolate the series instead of mean
                interpolated_value = interpolation_df.loc[:,column].mean()
            else: 
                # mean of nearby stations
                print(station, column, i)